In [27]:
import sys
import os

# Get the absolute path of the project root (adjust as needed)
project_root = os.path.abspath("../..")  # Moves one level up to project root

# Add to sys.path if not already present
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [1]:
%pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from qdrant_client import QdrantClient, models
import numpy as np
from rich import print
from utils import get_tokens

openai_api_key: EMPTY
openai_api_base: http://localhost:9000/v1
export_template_path: ./outflow/templates
qdrant_host: localhost
qdrant_port: 6333
neo4j_connection: bolt://localhost:7687
neo4j_user: neo4j
neo4j_password: ********


## Collection Setup

In [34]:
import ollama

# Initialize Ollama client
oclient = ollama.Client(host="localhost")
# Create a vector for the Component node
response = oclient.embeddings(model="nomic-embed-text", prompt="Hello, world")
embeddings = response["embedding"]
# 2. Convert to Qdrant sparse vectors
COLLECTION_NAME = "demo_collection"
client = QdrantClient(host="localhost", port="6333")

if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "text-dense": models.VectorParams(
            size=len(embeddings), distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(on_disk=False)
        )
    },
)

True

## Data Loading

In [4]:
from typing import List, Optional
from pydantic_xml import BaseXmlModel, attr, element


class NamePresentation(BaseXmlModel):
    edt: Optional[str] = attr("edt")
    value: str = ""


class Description(BaseXmlModel):
    edt: Optional[str] = attr("edt")
    value: str = ""


class ProductVariantConditionRef(BaseXmlModel):
    ref: Optional[str] = attr("ref")
    value: str = ""


class Server(BaseXmlModel):
    canAddress: str = element("CanAddress")
    propertyName: str = element("PropertyName")


class Property(BaseXmlModel):
    server: Server = element("Server")


class PropertyGroup(BaseXmlModel):
    name: str = element("Name")
    namePresentation: NamePresentation = element("NamePresentation")
    description: Optional[Description] = element("Description", default=None)  # ✅ Added
    property: List[Property] = element("Property")


class FunctionPropertyGroup(BaseXmlModel, tag="FunctionPropertyGroup"):
    name: str = element("Name")
    namePresentation: NamePresentation = element("NamePresentation")
    description: Optional[Description] = element("Description", default=None)
    productVariantConditionRef: Optional[ProductVariantConditionRef] = element("ProductVariantConditionRef", default=None)
    ufNumber: List[int] = element("UFNumber")

    propertyGroup: Optional[List[PropertyGroup]] = element("PropertyGroup", default=None)
    property: Optional[List[Property]] = element("Property", default=None)

    class Config:
        xml_ns = {
            "xsi": "http://www.w3.org/2001/XMLSchema-instance"
        }

In [5]:
from transformers import AutoTokenizer

# Load the Qwen tokenizer
tokenizer = AutoTokenizer.from_pretrained("NovaSky-AI/Sky-T1-32B-Flash", trust_remote_code=True)

/home/rajesh/Method_Engineer_Support_Tool/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
# load all the physical quantities from the ./data/Function-Parameters/PhysicalQuantity/ folder
from pathlib import Path

physical_quantity_files = Path("./data/Function-Parameters/FunctionViewAdjust").glob(
    "*.xml"
)

# make sure their names start with PhysicalQuantity
physical_quantity_files = [
    file
    for file in physical_quantity_files
    if file.name.startswith("FunctionPropertyGroup")
]

physical_quantities = []
token_count = 0
print("total files: ", len(physical_quantity_files))
funciton_properties_details = ""

function_group_objects = []

for file in physical_quantity_files:
    with file.open("rb") as f:
        xml_data = f.read()
        obj = FunctionPropertyGroup.from_xml(xml_data)
        physical_quantities.append(obj)

        model_json = obj.model_dump_json(indent=2)

        input_ids = tokenizer(model_json)["input_ids"]
        token_count += len(input_ids)

        funciton_properties_details += "\n\nFUNCTION PROPERTY GROUP " + obj.name + "\n"
        if obj.property:
            for prop in obj.property:
                funciton_properties_details += prop.server.propertyName + "\n"
        if obj.propertyGroup:
            for prop_group in obj.propertyGroup:
                funciton_properties_details += prop_group.name + "\n"
                for prop in prop_group.property:
                    funciton_properties_details += prop.server.propertyName + "\n"

        function_group_objects.append(obj)

print(f"Token count: {token_count}")

# tokens for all physical quantities
tokens_count_function_properties = len(
    tokenizer(funciton_properties_details)["input_ids"]
)

print(f"Token count for all function properties: {tokens_count_function_properties}")
# print(physical_quantities)

total files:  164

Token count: 82831

Token count for all function properties: 15219

In [7]:
# # 0. Sample data
# documents = [
#     "Reading Light",
#     "Rear Cam Disp",
#     "GPS Navigation",
#     "GPS Panel",
#     "Heated Seats",
#     "Read Lite",
# ]

# # 1. Fit TF-IDF vectorizer with char n-grams
# vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
# X_sparse = vectorizer.fit_transform(documents)

documents = []
for obj in function_group_objects:
    document = ""

    if obj.property:
        for prop in obj.property:
            document += prop.server.propertyName + "\n"
    if obj.propertyGroup:
        for prop_group in obj.propertyGroup:
            document += prop_group.name + "\n"
            for prop in prop_group.property:
                document += prop.server.propertyName + "\n"
    print(document)
    documents.append(document)

# # 1. Fit TF-IDF vectorizer with char n-grams
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
X_sparse = vectorizer.fit_transform(documents)


PVGLight
Approach_light

numberOfTanksBus
totalTankVolumeBus
numberOfTanksExtra
totalTankVolumeExtra

PTOGearDeac

Horn_Var
Horn_Source

OptionalOutputSignalsInC493
Out_1
Out_2
Out_3
Out_4
Out_5
Out_6
Out_1mode
Out_2mode
Out_3mode
Out_4mode
Out_5mode
Out_EngSpd
Out_Freq1
Out_Freq2
Out_Freq3
Out_Freq4
Out_Freq5
Out_Freq6
AdaptationToBodyworkGauge
OilPressure
EngineTemperature
FuelLevel
EngineLoad
CircuitPressure1
CircuitPressure2
TransOilTemp
RetOilTemp

...

SeatbeltReminder
SeatbeltReminderSound
SeatbeltRemind

AutomaticAuxiliaryBrakeSwitchSetting
AutomaticAuxilliaryBrakeSwitchSetting

BusStopBrakeSound
ActivateBusStopBrakeActiveLevel
AutoHold_AD_Sound
AutoHold_AD_Sound_2
SUSP_BusStopBrakeByKneeling
SUSP_BusStopBrakeByKneeling

Buzz_Sound
CAN18_BB_SoundConfig
Buzz_Source
ActivateBuzzerActiveLevel

EngineHatchOpenActiveLevel
HatchOpenActiveLevel

In [18]:
X_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 33651 stored elements and shape (164, 11807)>

In [85]:
def get_dense_vector(document):
    """
    Convert sparse vector to dense format.
    """
    response = oclient.embeddings(
        model="nomic-embed-text", prompt=" ".join(get_tokens(document))
    )
    embeddings = response["embedding"]

    return embeddings


# 3. Upsert documents
for idx in range(X_sparse.shape[0]):
    row = X_sparse[idx]
    obj = function_group_objects[idx]
    document = documents[idx]
    indices = row.indices
    values = row.data

    client.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(
                id=idx,
                payload={"json": obj.model_dump_json(), "document": document, "tokens": get_tokens(document)},
                vector={
                    "text-sparse": models.SparseVector(
                        indices=indices.tolist(),
                        values=values.tolist(),
                    ),
                    "text-dense": get_dense_vector(document),
                },
            )
        ],
    )

## Query

In [174]:
# 4. Query preparation
name="DHSC_AtEgegTyp"
description = """
Brake pedal behaviour for activating Automatic DHSC, See parameter for description.
"""
query_vec = vectorizer.transform([name])
query_indices = query_vec.indices
query_values = query_vec.data

dense_embedding = get_dense_vector(name + " " + description)

##### Sparse Query


In [175]:
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=models.SparseVector(
        indices=query_indices,
        values=query_values
    ),
    using="text-sparse",
    limit=3,
)

##### Dense Query

In [156]:
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=dense_embedding,
    using="text-dense",
)

##### Fusion Query

In [178]:
results = client.query_points(
    collection_name=COLLECTION_NAME,
    prefetch=[
        models.Prefetch(
            query=models.SparseVector(indices=query_indices, values=query_values),
            using="text-sparse",
            limit=20,
        ),
        models.Prefetch(
            query=dense_embedding,  # <-- dense vector
            using="text-dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
)

### Results

In [179]:
print(results)

QueryResponse(
    points=[
        ScoredPoint(
            id=43,
            version=207,
            score=1.0,
            payload={
                'json': '{"name":"DownhillSpeedControl","namePresentation":{"edt":"nfTxt","value":"Downhill speed 
control"},"description":null,"productVariantConditionRef":null,"ufNumber":[134],"propertyGroup":null,"property":[{"
server":{"canAddress":"27","propertyName":"DHSC_Variant"}},{"server":{"canAddress":"27","propertyName":"DHSC_Defaul
tOffset"}},{"server":{"canAddress":"10","propertyName":"DHSCOffset"}},{"server":{"canAddress":"27","propertyName":"
DHSC_MinOffset"}},{"server":{"canAddress":"10","propertyName":"DHSC"}},{"server":{"canAddress":"10","propertyName":
"DHSCMinOffset"}},{"server":{"canAddress":"10","propertyName":"DhscBrakeLight"}},{"server":{"canAddress":"DC","prop
ertyName":"DhscBrakeLight"}},{"server":{"canAddress":"DC","propertyName":"DHSCType"}},{"server":{"canAddress":"27",
"propertyName":"DHSC_AutoEngageType"}}]}',
                'document': 
'DHSC_Variant\nDHSC_DefaultOffset\nDHSCOffset\nDHSC_MinOffset\nDHSC\nDHSCMinOffset\nDhscBrakeLight\nDhscBrakeLight\
nDHSCType\nDHSC_AutoEngageType\n',
                'tokens': [
                    'dhs',
                    'c',
                    'variant',
                    'dhs',
                    'c',
                    'default',
                    'offset',
                    'dhs',
                    'c',
                    'offset',
                    'dhs',
                    'cm',
                    'in',
                    'offset',
                    'dhs',
                    'c',
                    'dhs',
                    'cm',
                    'in',
                    'offset',
                    'dhs',
                    'c',
                    'brake',
                    'light',
                    'dhs',
                    'c',
                    'brake',
                    'light',
                    'dhs',
                    'ctype',
                    'dhs',
                    'cau',
                    'to',
                    'engage',
                    'type'
                ]
            },
            vector=None,
            shard_key=None,
            order_value=None
        ),
        ScoredPoint(
            id=74,
            version=238,
            score=0.53333336,
            payload={
                'json': 
'{"name":"EngineSpeedControlUsingAcceleratorPedal","namePresentation":{"edt":"nfTxt","value":"Engine speed control 
using accelerator pedal\\n    
"},"description":null,"productVariantConditionRef":null,"ufNumber":[119],"propertyGroup":null,"property":[{"server"
:{"canAddress":"03","propertyName":"Kickdown"}},{"server":{"canAddress":"03","propertyName":"Load_Class_Kick_Down"}
},{"server":{"canAddress":"00","propertyName":"RegType"}},{"server":{"canAddress":"00","propertyName":"RqRqvSwitchF
unction"}},{"server":{"canAddress":"00","propertyName":"AccelerationLimit"}},{"server":{"canAddress":"00","property
Name":"AccLimiterEnable"}},{"server":{"canAddress":"00","propertyName":"AccLimitKickDownSpeed"}},{"server":{"canAdd
ress":"00","propertyName":"AccelerationLimitFunctionParameter"}},{"server":{"canAddress":"DC","propertyName":"Accel
eratorPedalLogic"}},{"server":{"canAddress":"DC","propertyName":"EmulatedIceBrakeTorque"}},{"server":{"canAddress":
"00","propertyName":"AlimVehSpeed-AccLimit"}}]}',
                'document': 
'Kickdown\nLoad_Class_Kick_Down\nRegType\nRqRqvSwitchFunction\nAccelerationLimit\nAccLimiterEnable\nAccLimitKickDow
nSpeed\nAccelerationLimitFunctionParameter\nAcceleratorPedalLogic\nEmulatedIceBrakeTorque\nAlimVehSpeed-AccLimit\n'
,
                'tokens': [
                    'kickdown',
                    'loadclass',
                    'kickdown',
                    'reg',
                    'type',
                    'r',
                    'qrq',
                    'v',
                    'switch',
            